In [1]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.remove(sys.path[0])

sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')
import get_transect_data
sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [2]:
path = "/Users/ajc/Core/simulations/barrow_icewedges/hydrologic_connection/ProjectionsO/"

In [3]:
#Dec 22, 2020: ProjectionsO
Simulations = [['CESM_C2%s'%i for i in range(3)], ['CESM_C1%s'%i for i in range(4)],['CESM_C3%s'%i for i in range(3)] ]# modified rain model, dessicated zone =0.1, longwaves


In [46]:
key_evap = 'surface-surface_subsurface_flux'
to_day_year_mm = (86400 * 365) *1000 
to_day_mm = (86400) *1000
to_year = 365
def getSurfaceData(i,loc_flag_, pos_):
    Data = []
    loc_flag = True
    for s in Simulations[i]:
        infile = os.path.join(path,s,'visdump_surface_data.h5')
        dx=h5py.File(infile,'r')
        
        if (loc_flag_):
            Dat11 = np.array(readdata.GetSurfVarFromVis(infile,key_evap,cellid=pos_)[key_evap])
            Dat11 = Dat11*to_day_mm
        else:
            Dat11 = np.array(readdata.GetSurfVarFromVis_All(infile,key_evap)[key_evap])
            
            Dat11 = Dat11.mean(axis=1)#*to_day_mm
            #print (Dat11)
            #break
        a1 = len(Dat11)%73
        a2 = len(Dat11) - a1
        
        Dat = Dat11[:a2]
        Data.append(Dat)
        #print (s, Dat)
    d = dict()
    d[key_evap] = np.concatenate(Data)
    #print (np.concatenate(Data))
    
    return d

In [47]:
d1 = getSurfaceData(0,loc_flag_=0, pos_=0)

In [48]:
plt.plot(d1[key_evap])

In [49]:
from multiprocessing import Pool
from functools import partial
num_processors = 3
pool=Pool(processes = num_processors)

def call_data(loc_flag,pos):
    global DataSurf1, DataSurf2, DataSurf3
    prod_x=partial(getSurfaceData, loc_flag_=loc_flag, pos_=pos)
    DataSurf1, DataSurf2, DataSurf3 = pool.map(prod_x,[i for i in range(len(Simulations))])
    Data = [DataSurf1, DataSurf2, DataSurf3]
    return Data

In [50]:
def get_summer_evap(d,evap_summer=0.5):
    x = []
    for dat in d:
        d1 = dat[dat>evap_summer] #if summer Evaporation is required than set evap_summer= 0.5
        x.append(np.mean(d1))
        
    #print (x)
    return x
    #d1 = d1.mean(axis=1)
    
def get_moving_average1(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < len1 - m:
            s1 = np.mean(d[i:i+m])
        else:
            s1 = np.mean(d[i:i+m])
        d_avg.append(s1)
    return d_avg

def get_moving_average(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < m:
            s1 = np.mean(d[:i+1])
                
        else:
            s1= np.mean(d[i-m:i])

        d_avg.append(s1)
    return d_avg

In [51]:
forcing_watershed_FM = h5py.File("/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2015-FM_2006.h5",'r')

In [52]:
d1 = forcing_watershed_FM ['precipitation rain [m s^-1]']
d2 = forcing_watershed_FM ['precipitation snow [m SWE s^-1]']
to_year_mm= 365*1000
rain_mm2 = np.reshape(d1,(-1,365)).mean(axis=1)*to_year_mm
snow_mm2 = np.reshape(d2,(-1,365)).mean(axis=1)*to_year_mm
precip_annual_watershed_FM = snow_mm2 + rain_mm2

In [53]:

#plt.plot(precip_annual_watershed_FM,'b')

In [88]:
%matplotlib qt
outfile='/Users/ajc/Documents/MyPapers/TroughHydrodynamics/figures/Dec22_uniform_om/'
def Plot_AvgEvap_Precip():
    fig, axs = plt.subplots(1,1, figsize=(6,3.5), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    
    plt.tight_layout()
    #axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Colors = ['b','k','r','r']
    Labels = ['zero head','specified head','dynamic head']
    T = ['a','b','c','d']
    Avg_change = []
    Data =call_data(loc_flag=False,pos=0)
    for i in range(3):
        d = np.reshape(Data[i][key_evap],(-1,73)).mean(axis=1)#*365
        #print (d)
        #break
        #d = get_summer_evap(d,evap_summer=0)
        d_avg5yr = get_moving_average(d,m=5)
        xx = np.linspace(2005,2100,95)
        axs.plot(xx,d_avg5yr,color=Colors[i],label=Labels[i]) #moving 5 year average
        #f = np.polyfit(xx,d_avg5yr,1)
        #p = np.poly1d(f)
        #axs.plot(xx,p(xx),color=Colors[i],linestyle='dashed')
    axs.set_ylim(-.001,0)
    #axs.set_xlim((2005, 2100))
    axs.set_xlabel('Year',fontsize=12, fontweight='normal')
    axs.set_ylabel('Mean annual ET [mm]',fontsize=12, fontweight='normal')
    axs.legend(loc='lower left',labelspacing=.1)
    plt.tight_layout()
    #plt.savefig(outfile+'ET-5yr_avg.pdf', bbox_inches='tight', dpi=500)
    plt.show()

In [89]:
Plot_AvgEvap_Precip()